In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add
from keras.models import Sequential
from keras.models import Model
import tensorflow as tf
import keras
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# seed the pseudorandom number generator
from random import seed
from random import random
# seed random number generator
seed(1)
# generate some random numbers
print(random(), random(), random())

In [ ]:
#load in the data
train_img = []
train_labels = []

test_img = []
test_labels = []

path_train = ('/kaggle/input/brain-tumor-classification-mri/Training/')
path_test = ('/kaggle/input/brain-tumor-classification-mri/Testing/')
img_size= 128

for i in os.listdir(path_train):
    for j in os.listdir(path_train+i):
        if i == "no_tumor":
            train_labels.append(0)
            train_img.append(cv2.cvtColor(cv2.resize(cv2.imread(path_train+i+'/'+j), (img_size,img_size)), cv2.COLOR_BGR2GRAY))
        else:
            if random() < 1:
                train_labels.append(1)
                train_img.append(cv2.cvtColor(cv2.resize(cv2.imread(path_train+i+'/'+j), (img_size,img_size)), cv2.COLOR_BGR2GRAY))


for i in os.listdir(path_test):
    for j in os.listdir(path_test+i):
        test_img.append(cv2.cvtColor(cv2.resize(cv2.imread(path_test+i+'/'+j), (img_size,img_size)), cv2.COLOR_BGR2GRAY))
        if i == "no_tumor":
            test_labels.append(0)
        else:
            test_labels.append(1)
        
train_img = (np.array(train_img))
test_img = (np.array(test_img))


In [ ]:
#possibly add data augmentation here to try to improve performance, decided on clahe to enhance image, and median filter to average out noise.
#canny was added as another preprocessing step for feature extraction
clahe_train = []
clahe_test = []

median_filter = np.ones((3,3),np.float32)/9
clahe = cv2.createCLAHE(clipLimit =2.0, tileGridSize=(8,8))
for i in range(len(train_img)):
    clahe_train.append(cv2.Canny(clahe.apply((cv2.filter2D(np.uint8(train_img[i]),-1,median_filter))), threshold1=100, threshold2=200))

train_img = np.array(clahe_train)

for i in range(len(test_img)):
    clahe_test.append(cv2.Canny(clahe.apply((cv2.filter2D(np.uint8(test_img[i]),-1,median_filter))), threshold1=100, threshold2=200))

test_img = np.array(clahe_test)



In [ ]:
#normalize images
train_img_normalized = train_img/255
test_img_normalized = test_img/255

In [ ]:
input_size = train_img_normalized.size/128/128

In [ ]:
#train and validation split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_img_normalized, np.array(train_labels), test_size=0.33, random_state=42)

In [ ]:
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

In [ ]:
#loop to vary svm paremeters to find a good fit
from sklearn import svm
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

kernel_type = ["linear", "poly", "rbf", "sigmoid"]
degree = [2, 3, 4, 5, 6, 7, 8, 9]
best_recall = 0.0
for i in range(len(kernel_type)):
    for j in range(len(degree)):
        clf = svm.SVC(kernel = kernel_type[i], degree = degree[j])
        clf.fit(X_train.reshape(int(X_train.size/128/128), 128*128), np.array(y_train))
        y_pred_test = clf.predict(X_val.reshape(int(X_val.size/128/128), 128*128))
        print(i)
        print(j)
        print('Model f1 score: {0:0.4f}'. format(f1_score(np.array(y_val), y_pred_test)))
        print('Model accurracy score: {0:0.4f}'. format(accuracy_score(np.array(y_val), y_pred_test)))i
        

In [ ]:
#final svm
clf = svm.SVC(kernel = "rbf", degree = 4)
clf.fit(X_train.reshape(int(X_train.size/128/128), 128*128), np.array(y_train))

In [ ]:
y_pred_test = clf.predict(test_img_normalized.reshape(int(test_img_normalized.size/128/128), 128*128))
print(y_pred_test)
from sklearn.metrics import accuracy_score
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(np.array(test_labels), y_pred_test)))
print('Model precision score: {0:0.4f}'. format(precision_score(np.array(test_labels), y_pred_test)))
print('Model recall score: {0:0.4f}'. format(recall_score(np.array(test_labels), y_pred_test)))
print('Model F1 score: {0:0.4f}'. format(f1_score(np.array(test_labels), y_pred_test)))

In [ ]:
#loop to vary RF paremeters to find a good fit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
#intial number of trees, best was 100 so decreasing range number_of_trees = [100, 250, 500, 750, 1000]
number_of_trees = [20, 40, 60, 80, 100]
#initial max depth was, 12 was best so putting more values around 12 max_depth = [6, 9, 12, 15, 18]
max_depth = [10,11, 12, 13, 14]
crit = ["gini", "entropy"]
best_recall = 0.0
for i in range(len(max_depth)):
    for j in range(len(number_of_trees)):
        for k in range(len(crit)):
            clf = RandomForestClassifier(max_depth=max_depth[i], criterion = crit[k], n_estimators = number_of_trees[j], random_state=0)
            clf.fit(X_train.reshape(int(X_train.size/128/128), 128*128), np.array(y_train))
            y_pred_test = clf.predict(X_val.reshape(int(X_val.size/128/128), 128*128))
            print(i)
            print(j)
            print(k)
            print('Model recall score: {0:0.4f}'. format(recall_score(np.array(y_val), y_pred_test)))
            print('Model accurracy score: {0:0.4f}'. format(accuracy_score(np.array(y_val), y_pred_test)))
        

In [ ]:
#from the loops above got optimal RF
clf = RandomForestClassifier(max_depth=12, criterion = "gini", n_estimators = 20, random_state=0)
clf.fit(train_img_normalized.reshape(int(train_img_normalized.size/128/128), 128*128), np.array(train_labels))

In [ ]:
y_pred_test = clf.predict(test_img_normalized.reshape(int(test_img_normalized.size/128/128), 128*128))
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(np.array(test_labels), y_pred_test)))
print('Model precision score: {0:0.4f}'. format(precision_score(np.array(test_labels), y_pred_test)))
print('Model recall score: {0:0.4f}'. format(recall_score(np.array(test_labels), y_pred_test)))
print('Model F1 score: {0:0.4f}'. format(f1_score(np.array(test_labels), y_pred_test)))

In [ ]:
#loop to vary KNN paremeters to find a good fit
from sklearn.neighbors import KNeighborsClassifier

number_of_neighbors = [2, 3, 4, 5, 6]
algo = ["auto", "ball_tree", "kd_tree", "brute"]
weights = ["uniform", "distance"]

for i in range(len(number_of_neighbors)):
    for j in range(len(algo)):
        for k in range(len(weights)):
            clf = KNeighborsClassifier(n_neighbors=number_of_neighbors[i], algorithm = algo[j], weights = weights[k])
            clf.fit(X_train.reshape(int(X_train.size/128/128), 128*128), np.array(y_train))
            y_pred_test = clf.predict(X_val.reshape(int(X_val.size/128/128), 128*128))
            print(i)
            print(j)
            print(k)
            print('Model recall score: {0:0.4f}'. format(recall_score(np.array(y_val), y_pred_test)))
            print('Model accurracy score: {0:0.4f}'. format(accuracy_score(np.array(y_val), y_pred_test)))

In [ ]:
#optimal KNN
clf = KNeighborsClassifier(n_neighbors=number_of_neighbors[1], algorithm = algo[2], weights = weights[1])
clf.fit(X_train.reshape(int(X_train.size/128/128), 128*128), np.array(y_train))

In [ ]:
y_pred_test = clf.predict(test_img_normalized.reshape(int(test_img_normalized.size/128/128), 128*128))
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(np.array(test_labels), y_pred_test)))
print('Model precision score: {0:0.4f}'. format(precision_score(np.array(test_labels), y_pred_test)))
print('Model recall score: {0:0.4f}'. format(recall_score(np.array(test_labels), y_pred_test)))
print('Model F1 score: {0:0.4f}'. format(f1_score(np.array(test_labels), y_pred_test)))

In [ ]:
#output confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
cm = confusion_matrix(np.array(test_labels), y_pred_test, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.show()
